In [ ]:
# Student one full name
Student1_Name = "Zeyad Tarek"

# Student two full name
Student2_Name = "Asmaa Adel"

# team ID
team_ID = ""

In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import bigrams, FreqDist

# Requirement Description
You are given pizza orders dataset. You are required to build some Language Models from the training set. You will then use these models for predicting the masked tokens in the test set. For each order in the test set, a random word was replaced with the word mask. Your task will be to predict the mask work. This requirement is divided into three parts:

## Part 1: Data Preprocessing
In this part, you will preprocess the given dataset. Once you have preprocessed the orders, you can then use them to build the language models.

## Part 2: Bi-gram Language Model
In this part, You will build a bi-gram language model from scratch. The Bi-gram language model need to have two main functions: train, predict.

## Part 3: RNN Language Model
In this part, You will build an RNN language model from scratch. The RNN language model need to have two main functions: train, predict.

---
Let's get started :D

In [ ]:
# Read the training and test sets
train_sentences = pd.read_csv('pizza_train.csv')['text'].to_list()
test_sentences = pd.read_csv('masked_pizza_test.csv')['text'].to_list()
print(f"train set size = {len(train_sentences)}")
print(f"test set size = {len(test_sentences)}")

# Part 1: Data Preproccessing
After reading the train and test sets You will do the following steps on both train and test sets:
Loop over the sentences to do the following
   1. convert all characters to lower (hint: lower())
   2. tokenize the sentence (hint: use word_tokenize())
   3. Add the start sentence \<s> and end sentence \</s> tokens at the beginning and end of each tokenized sentence
   4. Add the tokens of the sentence in a predefined set to collect the vocabulary

In [ ]:
# This function takes a List of sentences to preprocess them and vocabulary to extend
# It returns a list of sentences after preprocessing and the vocabulary it found
def preprocess(sentences, vocab):
    tokenized_sentences = []
    for i in range(len(sentences)):
        sentence = sentences[i]
        
        ################################ TODO: Preprocessing  ####################################
        # convert all characters to lower (hint: use lower())

        
        # tokenize the sentence (hint: use word_tokenize())

        
        # Add the start sentence <s> and end sentence </s> tokens at the beginning and end of each tokenized sentence

        
        # Add the tokens of the sentence in the predefined set vocab to collect the vocabulary

        
        # Add the sentence to the tokenized_sentences

        #########################################################################################  
    
    return tokenized_sentences, vocab

In [ ]:
vocab = set()
preprocessed_train_sentences, vocab = preprocess(train_sentences, vocab)
preprocessed_test_sentences, vocab = preprocess(test_sentences, vocab)
vocab.remove("mask")


assert type(preprocessed_train_sentences) == list, "Type of preprocessed train should be list"
assert type(preprocessed_test_sentences) == list, "Type of preprocessed test should be list"
assert len(preprocessed_train_sentences) == 10000, "The number of train sentences is not correct"
assert len(preprocessed_test_sentences) == 1000, "The number of train sentences is not correct"
assert len(vocab) == 307, "Error in the number of vocabulary"

# Part two:
In this part, You will build an Add one (La place) Smoothed bi-gram language model

In [ ]:
class BigramLM:
    
    # The class constructor takes the vocabulary
    # creates a |V| * |V| numpy 2D matrix for the bi-gram model filled with zeros
    # It also creates two dictionaries to be used for token identification
    def __init__(self, vocab):
        self.id2word = {i: word for i, word in enumerate(list(vocab))}
        self.word2id = {word: i for i, word in self.id2word.items()}
        self.vocab_size = len(vocab)
    
        # create a numpy |V| * |V| 2D array filled with zeros
        self.CountsMatrix = np.zeros((self.vocab_size, self.vocab_size), dtype=int)
        
    
    ####################################### TODO: Complete the BigramLM class ######################################
    # This function is responsible for training the Language Model
    # It is given the preprocessed training set sentences
    # The goal is to fill the 2D matrix with the appropriate counts
    # hint: check bigrams() and FreqDist() from nltk
    # hint: loop over the sentences to fill the matrix with the appropriate counts
    def train(self, train_sentences):
        pass
                
    
    # this function takes two words and calculates the Add-one Smoothed bi-gram probability of it
    # Of course the function will make use of the 2D counts matrix built while training
    # The function assumes that word1 precedes word2
    # The function must return the calculated probability
    def calcProbability(self, word1, word2):
        pass
    
    
    # This function takes a preprocessed tokenized sentence with exactly one token = mask (look at the masked test set)
    # The function returns a word from the vocabulary that is more likely to be masked
    # hint: calculate the probabilities of all possible words in the vocabulary and return the word with the maximum probability
    def predict(self, tokenized_sentence):
        pass
    
    
    # This function takes a token and samples the next token
    # hint: check np.random.choice
    def sample(self, token):
        pass
    
    
    # This function generates a new order using the Bi-gram probabilities
    # Returns one string where the generated tokens are white space separated
    # Note: The start token <s> and end token </s> shouldn't appear in the generated order
    # hint: start the generation with <s> and stop generating tokens when you reach </s>
    def generateOrder(self):
        pass
    ################################################################################################################

# The next cell to make sure your Bigram LM is correct

In [ ]:
bigramLM = BigramLM(vocab)
bigramLM.train(preprocessed_train_sentences)

assert bigramLM.calcProbability('i', "'d") == 0.6469576435175232, "Probability Error"
assert bigramLM.calcProbability("'d", "i") == 0.00030165912518853697, "Probability Error"
assert bigramLM.predict(['<s>', 'one', 'mask', 'sized', 'green', 'pepper', '</s>']) == 'party', "Prediction Error"

# Part three:
In this part, you will build and RNN Language model from scratch. This part may be a little bit tricky specially in the back propagation, but we will go trough it step by step

In [ ]:
class RNNLM:
    
    # The class constructor takes the vocabulary
    # creates two dictionaries to be used for token identification
    # creates the embeddings matrix (one hot encodings for tokens)
    # creates all wights of the RNN Wx, Wh, b, Wy, by
    def __init__(self, vocab, hidden_dim=128):
        vocab.add('null')
        self.id2word = {i: word for i, word in enumerate(list(vocab))}
        self.word2id = {word: i for i, word in self.id2word.items()}
        self.vocab_size = len(vocab)
        self.null_word_index = self.word2id['null']
        
        # Create a numpy |V| * |V| 2D identity array
        self.embeddings = np.identity(self.vocab_size)
        
        # Initialize RNN weights
        np.random.seed(5)
        self.Wx = np.random.randn(self.vocab_size, hidden_dim).astype(np.float64)
        self.Wx /= np.sqrt(self.vocab_size)
        self.Wh = np.random.randn(hidden_dim, hidden_dim).astype(np.float64)
        self.Wh /= np.sqrt(hidden_dim)
        self.b = np.zeros(hidden_dim).astype(np.float64)
        self.Wy = np.random.randn(hidden_dim, self.vocab_size).astype(np.float64)
        self.Wy /= np.sqrt(hidden_dim)
        self.by = np.zeros(self.vocab_size).astype(np.float64)

        
    def one_step_forward(self, x, h_prev, Wx, Wh, b):
        """
        This function runs one time step of the RNN. It should implement the RNN equation
        that takes input embedding and the previous hidden state then produces the new hidden state.
        
        Inputs:
        - x: Input data of the current time step of shape (N, D) 
             where N is the number of tokens and D is the vocab size
        - h_prev: the hidden state from the previous time step of shape (N, H)
        - Wx: the weight matrix for input to hidden transformation of shape (D, H)
        - Wh: the weight matrix for hidden to hidden tranformation of shape (H, H)
        - b: the bias of shape (H,)
        
        Returns a tuple of:
        - next_h: Next hidden state of shape (N, H)
        - cache: a tupple of all data needed for backpropagation
        """
        next_h, cache = None, None
        
        ############################### TODO: Impelement RNN equation for forward pass ##############################
        
        # use numpy to implement this equation next_h = tanh(Wx*x + Wh*h_prev + b)

        
        # cache all needed values for backpropagation
        
        
        #############################################################################################################
        
        return next_h, cache
    
    
    def one_step_backward(self, dnext_h, cache):
        """
        This function implements the backward pass of a single time step RNN
        
        Inputs:
        - dnext_h: the gradient of the loss with respect to the next hidden state of shape (N, H)
        - cache: a tupple that we cached before from the forward pass
        
        Returns:
        - dprev_h: Gradients of previous hidden state, of shape (N, H)
        - dWx: Gradients of input-to-hidden weights, of shape (D, H)
        - dWh: Gradients of hidden-to-hidden weights, of shape (H, H)
        - db: Gradients of bias vector, of shape (H,)
        """
        dprev_h, dWx, dWh, db = None, None, None, None
        x, prev_h, Wx, Wh, b, tanh = cache
        
        ################# TODO: Impelement the backward pass by calculating derivatives and applying chain rule ###########

        
        
        ###################################################################################################################
        
        return dprev_h, dWx, dWh, db
    
    
    def full_forward_pass(self, x, h0, Wx, Wh, b):
        """
        This function runs the RNN forward on an entire sequence of data. We assume an input
        sequence composed of T vectors, each of dimension D. The RNN uses a hidden
        size of H, and we work over a minibatch containing N sequences. After running
        the RNN forward, we return the hidden states for all timesteps.
        
        Inputs:
        - x: Input data for the entire timeseries, of shape (N, T, D).
        - h0: Initial hidden state, of shape (N, H)
        - Wx: Weight matrix for input to hidden transformation, of shape (D, H)
        - Wh: Weight matrix for hidden to hidden transformation, of shape (H, H)
        - b: Biases of shape (H,)
        
        Returns a tuple of:
        - h: Hidden states for the entire timeseries, of shape (N, T, H).
        - cache: All values needed in the backward pass
        """
        h, cache = None, None
        N, T, D = x.shape
        H = h0.shape[1]
        #################### TODO: Implement the full forward pass of the RNN ###########################
        # Hint 1: You will use the one_step_forward to compute each time step hidden state and cache
        # Hint 2: For loop over T and call one_step_forward with the appropriate parameters
        
        # create numpy array filled with zeros of shape (N, T, H) for h to be filled then returned
        
        
        # initialize the cache as an empty list to be filled with all caches returned from one_step_forward
        
        
        # loop over T and calculate the needed hidden states and caches
        
        
        #################################################################################################
        
        return h, cache
    
    
    def full_backward_pass(self, dh, cache):
        """
        This function computes the backward pass for the RNN over an entire sequence of data.
        Inputs:
        - dh: The loss gradient with respect to all hidden states, of shape (N, T, H). 

        Returns a tuple of:
        - dh0: Gradient of initial hidden state, of shape (N, H)
        - dWx: Gradient of input to hidden weights, of shape (D, H)
        - dWh: Gradient of hidden to hidden weights, of shape (H, H)
        - db: Gradient of biases, of shape (H,)
        """
        dh0, dWx, dWh, db = None, None, None, None
        N, T, H = dh.shape
        
        #################### TODO: Implement the full backward pass of the RNN ###########################
        # Hint 1: You will use the one_step_backward to compute each time step gradients
        # Hint 2: For loop over T and call one_step_backward with the appropriate parameters
        # Hint 3: Don't forget to add dprev_h returned from one_step_backward to the appropriate index in dh while passing
        #         to one_step_backward
        
        
        ##################################################################################################
        
        return dh0, dWx, dWh, db
    
    def hidden_to_scores_forward(self, h, Wy, by):
        """
        This function implements the forward pass of transforming the hidden state to the scores vector.
        The input is a set of H-dimensional vectors arranged into a minibatch of N timeseries, each of length T.
        The goal is transform from the hidden state to the scores output vector with size V (number of vocabulary).
        
        Inputs:
        - h: Input data of shape (N, T, H)
        - Wy: Weights of shape (H, V)
        - by: Biases of shape (V,)
       
        Returns a tuple of:
        - y: Output data of shape (N, T, V)
        - cache: Values needed for the backward pass
        """
        N, T, H = h.shape
        V = by.shape[0]
        
        y, cache = None, None
        
        ##################### TODO: calculate the scores matrix y and cache all needed values ########################
        # Hint 1: you need to caculate this formula y = Wy*h + by
        # Hint 2: you can reshape h from (N, T, H) to (N*T, H) before the matrix multiplication
        #         then reshape it back to (N, T, H) after the matrix multiplication and before adding the by
        
        
        ##############################################################################################################
        
        return y, cache
    
    
    def hidden_to_scores_backward(self, dy, cache):
        """
        Backward pass for the hidden to scores layer.
        Input:
        - dy: The gradients of shape (N, T, V)
        - cache: Values from forward pass
        Returns a tuple of:
        - dh: Gradient of input, of shape (N, T, H)
        - dWy: Gradient of weights, of shape (H, V)
        - dby: Gradient of biases, of shape (V,)
        """
        h, Wy, by, y = cache
        N, T, H = h.shape
        V = by.shape[0]

        dh, dWy, dby = None, None, None
        
        ################################ TODO: calculate the gradients ##############################
        # Hint: use reshape to ensure correct dimensions for matrix multiplications and derivat
        
        #############################################################################################

        return dh, dWy, dby
    
    def softmax_loss(self, x, y, mask):
        """
        This function calculates the softmax loss over minibatches of size N.
        
        Inputs:
        - x: Input scores, of shape (N, T, V)
        - y: Ground-truth indices, of shape (N, T) where each element is in the range
             0 <= y[i, t] < V
        - mask: Boolean array of shape (N, T) where mask[i, t] tells whether or not
          the scores at x[i, t] should contribute to the loss.
        Returns a tuple of:
        - loss: Scalar giving loss
        - dx: Gradient of loss with respect to scores x, of shape (N, T, V).
        """
        N, T, V = x.shape
        x_flat = x.reshape(N * T, V)
        y_flat = y.reshape(N * T)
        mask_flat = mask.reshape(N * T)
        probs = np.exp(x_flat - np.max(x_flat, axis=1, keepdims=True))
        probs /= np.sum(probs, axis=1, keepdims=True)
        loss = -np.sum(mask_flat * np.log(probs[np.arange(N * T), y_flat])) / N
        dx_flat = probs.copy()
        dx_flat[np.arange(N * T), y_flat] -= 1
        dx_flat /= N
        dx_flat *= mask_flat[:, None]
        dx = dx_flat.reshape(N, T, V)
        return loss, dx
    
    
    def word_embeddings_forward(self, x_sentences, W_embedd):
        """
        This function takes input with tokens as indexes to produce their word embeddings
        
        Inputs:
        - x_sentences: input sentences of shape (N, T)
        - W_embedd: embeddings matrix of size (V, V)
        
        Returns:
        - x: matrix with word embeddings of shape (N, T, V)
        """
        return W_embedd[x_sentences]
    
    
    def loss(self, sentences):
        """
        This function computes the loss for training and the gradients of all RNN weights.
        
        Inputs:
        - sentences: Ground-truth sentences; an integer array of shape (N, T) where
          each element is in the range 0 <= y[i, t] < V
        Returns a tuple of:
        - loss: Scalar loss
        - grads: Dictionary of gradients for all RNN weights
        """
        # Since we are training a language model, for each token input to the RNN we should predict the
        # next token. So the inputs will the all tokens from begin to end -1 and the outputs will be all tokens
        # except the first token
        sentences_in = sentences[:, :-1]
        sentences_out = sentences[:, 1:]

        # You'll need this
        mask = (sentences_out != self.null_word_index)

        # Word embedding matrix
        W_embed = self.embeddings

        # Input-to-hidden, hidden-to-hidden, and biases for the RNN
        Wx, Wh, b = self.Wx, self.Wh, self.b

        # Weight and bias for the hidden-to-vocab transformation.
        Wy, by = self.Wy, self.by
        
        # sizes
        N, T = sentences.shape
        V, H = Wx.shape

        loss, grads = 0.0, {'Wx':None, 'Wh':None, 'b':None, 'Wy':None, 'by':None}
        
        ###################### TODO: Implement the forward and backward passes for the RNN LM #########################
        # In the forward pass you will need to do the following:                   
        
        # (1) Initilize the initial hidden state h0 to a zeros matrix of size (N, H)
        
        
        # (2) Use a word embedding layer to transform the words in captions_in  
        #     from indices to vectors, giving an array of shape (N, T, W).
        
        
        # (3) Call rnn_forward with the appropriate parameters to produce
        #     array of hidden states with shape (N, T, H)
        
             
        # (4) Call hidden_to_scores_forward to get an array of scores of shape (N, T, V)
         
        
        # (5) Call softmax_loss to compute loss using captions_out, ignoring
        #     the points where the output word is <NULL> using the mask above.
        
        
        
        # In the backward pass you will need to compute the gradient of the loss
        # with respect to all model parameters. Use the loss and grads variables
        # defined above to store loss and gradients; grads['k'] should give the 
        # gradients for self..

        # (1) Call hidden_to_scores_backward
        

        # (2) Call full_backward_pass
        

        # (3) save the gradients in the grads dictionary
        
        
        ###############################################################################################################

        return loss, grads
    
    def preprocessDataset(self, train_set):
        train_set = [[self.word2id[word] for word in s] for s in train_set]
        max_length = np.max([len(s) for s in train_set])
        train_set = [s + [self.null_word_index] * (max_length - len(s)) for s in train_set]
        return np.array(train_set, dtype=int)
    
    def train(self, sentences, batch_size=64, num_iterations=100, lr=0.0001):     
        for iteration in range(num_iterations):
            index = 0
            while index < len(sentences):
                loss, grads = self.loss(sentences[index: index + batch_size])
                self.Wx -= lr * grads['Wx']
                self.Wh -= lr * grads['Wh']
                self.b -= lr * grads['b']
                self.Wy -= lr * grads['Wy']
                self.by -= lr * grads['by']
                index += batch_size
            print(f"Iteration: {iteration} | loss = {loss}")
            
    def predict(self, tokenized_sentence):
        idx = tokenized_sentence.index("mask")
        prev_h = np.zeros((1, self.Wh.shape[0]), dtype=np.float64)
        x = self.word_embeddings_forward([self.word2id[token] for token in tokenized_sentence[:idx]], self.embeddings).reshape(1, idx, self.vocab_size)
        for i in range(idx):
            prev_h, cache_h = self.one_step_forward(x[:,i,:], prev_h, self.Wx, self.Wh, self.b)
        out, cache_voc = self.hidden_to_scores_forward(prev_h.reshape(1, 1, -1), self.Wy, self.by)
        return self.id2word[np.argmax(out)]
    
    
    def sample(self, token, prev_h):
        """
        This function takes a token and previous hidden state and samples the next token based on the softmax
        Probability distribution
        
        Inputs:
        - token: string containing the current token
        - prev_h: the previous hidden state of shape (1, H)
        
        Returns:
        - next_token: string containing the sampled next token
        - curr_h: the produced hidden state from the RNN
        """
        x = self.word_embeddings_forward([self.word2id[token]], self.embeddings).reshape(1, 1, self.vocab_size)
        Wx, Wh, b = self.Wx, self.Wh, self.b
        Wy, by = self.Wy, self.by
        
        next_token, curr_h = None, None
        ############################ TODO: implement the sampling ############################################
        # Call one_step_forward with the appropriate parameters
        
        # Call hidden_to_scores_forward to produce the scores
        
        # Compute the softmax your self. Hint: check np.exp
        
        # Sample the token using np.random.choice
        
        ######################################################################################################
        return next_token, curr_h
    
    def generateOrder(self):
        prev_h = np.zeros((1, self.Wh.shape[0]), dtype=np.float64)
        tokens = []
        t = '<s>'
        while t != '</s>':
            t, prev_h = self.sample(t, prev_h)
            tokens.append(t)
        return ' '.join(tokens[:-1])

# Let's Test Your Code
The next code cell implements functions for testing

In [ ]:
def rel_error(x, y):
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))
def eval_numerical_gradient(f, x, verbose=True, h=0.00001):
    """
    a naive implementation of numerical gradient of f at x
    - f should be a function that takes a single argument
    - x is the point (numpy array) to evaluate the gradient at
    """

    fx = f(x) # evaluate function value at original point
    grad = np.zeros_like(x)
    # iterate over all indexes in x
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:

        # evaluate function at x+h
        ix = it.multi_index
        oldval = x[ix]
        x[ix] = oldval + h # increment by h
        fxph = f(x) # evalute f(x + h)
        x[ix] = oldval - h
        fxmh = f(x) # evaluate f(x - h)
        x[ix] = oldval # restore

        # compute the partial derivative with centered formula
        grad[ix] = (fxph - fxmh) / (2 * h) # the slope
        if verbose:
            print(ix, grad[ix])
        it.iternext() # step to next dimension

    return grad


def eval_numerical_gradient_array(f, x, df, h=1e-5):
    """
    Evaluate a numeric gradient for a function that accepts a numpy
    array and returns a numpy array.
    """
    grad = np.zeros_like(x)
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        ix = it.multi_index

        oldval = x[ix]
        x[ix] = oldval + h
        pos = f(x).copy()
        x[ix] = oldval - h
        neg = f(x).copy()
        x[ix] = oldval

        grad[ix] = np.sum((pos - neg) * df) / (2 * h)
        it.iternext()
    return grad

# Test One Step Forward

In [ ]:
N, D, H = 3, 10, 4
rnnLM = RNNLM(vocab)
x = np.linspace(-0.4, 0.7, num=N*D).reshape(N, D)
prev_h = np.linspace(-0.2, 0.5, num=N*H).reshape(N, H)
Wx = np.linspace(-0.1, 0.9, num=D*H).reshape(D, H)
Wh = np.linspace(-0.3, 0.7, num=H*H).reshape(H, H)
b = np.linspace(-0.2, 0.4, num=H)

next_h, _ = rnnLM.one_step_forward(x, prev_h, Wx, Wh, b)
expected_next_h = np.asarray([
  [-0.58172089, -0.50182032, -0.41232771, -0.31410098],
  [ 0.66854692,  0.79562378,  0.87755553,  0.92795967],
  [ 0.97934501,  0.99144213,  0.99646691,  0.99854353]])

assert rel_error(expected_next_h, next_h) < 1e-8, "Error in one_step_forward"
print('next_h error: ', rel_error(expected_next_h, next_h))

# Test One Step Backwad

In [ ]:
rnnLM = RNNLM(vocab)
np.random.seed(231)
N, D, H = 4, 5, 6
x = np.random.randn(N, D)
h = np.random.randn(N, H)
Wx = np.random.randn(D, H)
Wh = np.random.randn(H, H)
b = np.random.randn(H)

out, cache = rnnLM.one_step_forward(x, h, Wx, Wh, b)

dnext_h = np.random.randn(*out.shape)

fx = lambda x: rnnLM.one_step_forward(x, h, Wx, Wh, b)[0]
fh = lambda prev_h: rnnLM.one_step_forward(x, h, Wx, Wh, b)[0]
fWx = lambda Wx: rnnLM.one_step_forward(x, h, Wx, Wh, b)[0]
fWh = lambda Wh: rnnLM.one_step_forward(x, h, Wx, Wh, b)[0]
fb = lambda b: rnnLM.one_step_forward(x, h, Wx, Wh, b)[0]

dx_num = eval_numerical_gradient_array(fx, x, dnext_h)
dprev_h_num = eval_numerical_gradient_array(fh, h, dnext_h)
dWx_num = eval_numerical_gradient_array(fWx, Wx, dnext_h)
dWh_num = eval_numerical_gradient_array(fWh, Wh, dnext_h)
db_num = eval_numerical_gradient_array(fb, b, dnext_h)

dprev_h, dWx, dWh, db = rnnLM.one_step_backward(dnext_h, cache)

assert rel_error(dprev_h_num, dprev_h) < 1e-9, "Error in one_step_backward"
assert rel_error(dWx_num, dWx) < 1e-9, "Error in one_step_backward"
assert rel_error(dWh_num, dWh) < 1e-9, "Error in one_step_backward"
assert rel_error(db_num, db) < 1e-9, "Error in one_step_backward"
print('dprev_h error: ', rel_error(dprev_h_num, dprev_h))
print('dWx error: ', rel_error(dWx_num, dWx))
print('dWh error: ', rel_error(dWh_num, dWh))
print('db error: ', rel_error(db_num, db))

# Test full forward pass

In [ ]:
N, T, D, H = 2, 3, 4, 5
rnnLM = RNNLM(vocab)

x = np.linspace(-0.1, 0.3, num=N*T*D).reshape(N, T, D)
h0 = np.linspace(-0.3, 0.1, num=N*H).reshape(N, H)
Wx = np.linspace(-0.2, 0.4, num=D*H).reshape(D, H)
Wh = np.linspace(-0.4, 0.1, num=H*H).reshape(H, H)
b = np.linspace(-0.7, 0.1, num=H)

h, _ = rnnLM.full_forward_pass(x, h0, Wx, Wh, b)
expected_h = np.asarray([
  [
    [-0.42070749, -0.27279261, -0.11074945,  0.05740409,  0.22236251],
    [-0.39525808, -0.22554661, -0.0409454,   0.14649412,  0.32397316],
    [-0.42305111, -0.24223728, -0.04287027,  0.15997045,  0.35014525],
  ],
  [
    [-0.55857474, -0.39065825, -0.19198182,  0.02378408,  0.23735671],
    [-0.27150199, -0.07088804,  0.13562939,  0.33099728,  0.50158768],
    [-0.51014825, -0.30524429, -0.06755202,  0.17806392,  0.40333043]]])
assert rel_error(expected_h, h) < 1e-7, "Error in full forward pass"
print('h error: ', rel_error(expected_h, h))

# Test full backward pass

In [ ]:
np.random.seed(231)
rnnLM = RNNLM(vocab)
N, D, T, H = 2, 3, 10, 5

x = np.random.randn(N, T, D)
h0 = np.random.randn(N, H)
Wx = np.random.randn(D, H)
Wh = np.random.randn(H, H)
b = np.random.randn(H)

out, cache = rnnLM.full_forward_pass(x, h0, Wx, Wh, b)

dout = np.random.randn(*out.shape)

dh0, dWx, dWh, db = rnnLM.full_backward_pass(dout, cache)

fx = lambda x: rnnLM.full_forward_pass(x, h0, Wx, Wh, b)[0]
fh0 = lambda h0: rnnLM.full_forward_pass(x, h0, Wx, Wh, b)[0]
fWx = lambda Wx: rnnLM.full_forward_pass(x, h0, Wx, Wh, b)[0]
fWh = lambda Wh: rnnLM.full_forward_pass(x, h0, Wx, Wh, b)[0]
fb = lambda b: rnnLM.full_forward_pass(x, h0, Wx, Wh, b)[0]

dh0_num = eval_numerical_gradient_array(fh0, h0, dout)
dWx_num = eval_numerical_gradient_array(fWx, Wx, dout)
dWh_num = eval_numerical_gradient_array(fWh, Wh, dout)
db_num = eval_numerical_gradient_array(fb, b, dout)

assert rel_error(dh0_num, dh0) < 1e-9, "Error in full_backward_pass"
assert rel_error(dWx_num, dWx) < 1e-9, "Error in full_backward_pass"
assert rel_error(dWh_num, dWh) < 1e-9, "Error in full_backward_pass"
assert rel_error(db_num, db) < 1e-9, "Error in full_backward_pass"

print('dh0 error: ', rel_error(dh0_num, dh0))
print('dWx error: ', rel_error(dWx_num, dWx))
print('dWh error: ', rel_error(dWh_num, dWh))
print('db error: ', rel_error(db_num, db))

# Check hidden to scores forward and backward

In [ ]:
# Gradient check for temporal affine layer
N, T, D, M = 2, 3, 4, 5
rnnLM = RNNLM(vocab)
x = np.random.randn(N, T, D)
w = np.random.randn(D, M)
b = np.random.randn(M)

out, cache = rnnLM.hidden_to_scores_forward(x, w, b)

dout = np.random.randn(*out.shape)

fx = lambda x: rnnLM.hidden_to_scores_forward(x, w, b)[0]
fw = lambda w: rnnLM.hidden_to_scores_forward(x, w, b)[0]
fb = lambda b: rnnLM.hidden_to_scores_forward(x, w, b)[0]

dx_num = eval_numerical_gradient_array(fx, x, dout)
dw_num = eval_numerical_gradient_array(fw, w, dout)
db_num = eval_numerical_gradient_array(fb, b, dout)

dx, dw, db = rnnLM.hidden_to_scores_backward(dout, cache)

assert rel_error(dx_num, dx), "Error in hidden to scores"
assert rel_error(dw_num, dw), "Error in hidden to scores"
assert rel_error(db_num, db), "Error in hidden to scores"

print('dx error: ', rel_error(dx_num, dx))
print('dw error: ', rel_error(dw_num, dw))
print('db error: ', rel_error(db_num, db))

# Check the loss function

In [ ]:
N, H = 5, 20
test_vocab = {'', 'cat', 'dog'}
V = len(test_vocab)
T = 10

model = RNNLM(test_vocab, H)

sentences_test = (np.arange(N * T) % V).reshape(N, T)

loss, grads = model.loss(sentences_test)


f = lambda _: model.loss(sentences_test)[0]
param_grad_num = eval_numerical_gradient(f, model.Wx, verbose=False, h=1e-6)
e = rel_error(param_grad_num, grads['Wx'])
print('%s relative error: %e' % ('Wx', e))
assert e < 1e-6, "Error in dWx"

param_grad_num = eval_numerical_gradient(f, model.Wh, verbose=False, h=1e-6)
e = rel_error(param_grad_num, grads['Wh'])
print('%s relative error: %e' % ('Wh', e))
assert e < 1e-6, "Error in dWh"

param_grad_num = eval_numerical_gradient(f, model.b, verbose=False, h=1e-6)
e = rel_error(param_grad_num, grads['b'])
print('%s relative error: %e' % ('b', e))
assert e < 1e-6, "Error in db"

param_grad_num = eval_numerical_gradient(f, model.Wy, verbose=False, h=1e-6)
e = rel_error(param_grad_num, grads['Wy'])
print('%s relative error: %e' % ('Wy', e))
assert e < 1e-4, "Error in dWy"

param_grad_num = eval_numerical_gradient(f, model.by, verbose=False, h=1e-6)
e = rel_error(param_grad_num, grads['by'])
print('%s relative error: %e' % ('by', e))
assert e < 1e-6, "Error in dby"

# The following cells will produce the output files

In [ ]:
with open(f'{team_ID}.txt', 'w') as f:
    f.write(team_ID + '\n')
    f.write(Student1_Name + '\n')
    f.write(Student2_Name + '\n')

In [ ]:
bigramLM = BigramLM(vocab)
bigramLM.train(preprocessed_train_sentences)

predictions_bigram = []
for sentence in preprocessed_test_sentences:
    predictions_bigram.append(bigramLM.predict(sentence))
    
pd.DataFrame({'predictions': predictions_bigram}).to_csv(f'{team_ID}_bigram_predictions.csv', index=False)

In [ ]:
rnnLM = RNNLM(vocab)
train_set_RNN = rnnLM.preprocessDataset(preprocessed_train_sentences)
rnnLM.train(train_set_RNN, lr=0.01, num_iterations=20)

predictions_rnn = []
for sentence in preprocessed_test_sentences:
    predictions_rnn.append(rnnLM.predict(sentence))
    
pd.DataFrame({'predictions': predictions_rnn}).to_csv(f'{team_ID}_rnn_predictions.csv', index=False)

In [ ]:
gold_labels = pd.read_csv('labels.csv')
bigram_predictions = pd.read_csv(f'{team_ID}_bigram_predictions.csv')
rnn_predictions = pd.read_csv(f'{team_ID}_rnn_predictions.csv')

bigram_acc = np.where(bigram_predictions['predictions'] == gold_labels['labels'], True, False)
bigram_acc = np.sum(bigram_acc) / len(bigram_acc)

rnn_acc = np.where(rnn_predictions['predictions'] == gold_labels['labels'], True, False)
rnn_acc = np.sum(rnn_acc) / len(rnn_acc)

print("Bigram prediction Accuracy = ", bigram_acc)
print("RNN prediction Accuracy = ", rnn_acc)

# For your fun

In [ ]:
bigram_generation = []
np.random.seed(7)
for _ in range(10):
    s = bigramLM.generateOrder()
    bigram_generation.append(s)
    print(s)
    print()
    
pd.DataFrame({'generations': bigram_generation}).to_csv(f'{team_ID}_bigram_generations.csv', index=False)

In [ ]:
rnn_generation = []
np.random.seed(7)
for _ in range(10):
    s = rnnLM.generateOrder()
    rnn_generation.append(s)
    print(s)
    print()
    
pd.DataFrame({'generations': rnn_generation}).to_csv(f'{team_ID}_rnn_generations.csv', index=False)

# Thank you for your efforts :D